In [4]:
import folium
import matplotlib.cm as cm
import matplotlib.colors as cl

import openrouteservice as ors

## SETUP

# define colors
def color(value, maxValue):
    colors = ['#e31c00', '#aa5500', '#718e00', '#39c600', '#00ff00']
    N = len(colors)
    binSize = maxValue / N
    
    for i in range(N):
        if value < binSize*(i+1):
            return colors[i]
        
# set up ors client connected to local instance
host = "http://localhost:8082/ors"
profile = "driving-car"
client = ors.Client(base_url=host)

## NODE VISUALIZATION

# create map centered on Heidelberg
node_map = folium.Map(location = [49.411085, 8.685894], zoom_start = 13)

# set up bounding box.
# ORS expects [lon, lat]; Folium expects [lat, lon]
neuenheim = [[8.677139,49.412872],[8.690443,49.421080]]
neuenheim_folium = [list(reversed(coord)) for coord in neuenheim]

# visualize bounding box
folium.Rectangle(bounds=neuenheim_folium, color='#fcba03', fill=True, fill_color='#fcba03').add_to(node_map)

# Query node centrality
nodeResp = client.request(url = f'/v2/centrality/{profile}/json', get_params = {}, post_json = {'bbox': neuenheim})

# dict of nodeId -> score for easier access
nodeScores = {node['nodeId']: node['score'] for node in nodeResp['nodeScores']}

# determine max score for appropriate colors
maxScore = max(nodeScores.values())

# display nodes on map
for node in nodeResp['locations']:
    nodeId = node['nodeId']
    folium.CircleMarker(location = list(reversed(node['coord'])),
                        radius = 2,
                        color=color(nodeScores[nodeId], maxScore),
                        tooltip = f"id: {nodeId}, score: {nodeScores[nodeId]}").add_to(node_map)

node_map

In [8]:
## EDGE VISUALIZATION
from math import atan2, pi

# create map
edge_map = folium.Map(location = [49.411085, 8.685894], zoom_start = 13)

# set up bounding box.
# ORS expects [lon, lat]; Folium expects [lat, lon]
neuenheim = [[8.677139,49.412872],[8.690443,49.421080]]
neuenheim_folium = [list(reversed(coord)) for coord in neuenheim]

folium.Rectangle(bounds=neuenheim_folium, color='#fcba03', fill=True, fill_color='#fcba03').add_to(edge_map)

# we need a small deviation in the middle to differentiate between edges since this is a directed graph.
def midOffset(fromCoord, toCoord):
    # calculate midpoint
    mid = [sum(x)/2 for x in zip(fromCoord, toCoord)]
    
    # calculate x and y difference
    dx = fromCoord[0] - toCoord[0]
    dy = fromCoord[1] - toCoord[1]
    
    # deviate by 5% of orig. length
    norm = [dy*.05, -dx*.05]
    
    # calculate angle to draw arrow
    rot = atan2(dx, dy) * 180 / pi
    
    return [sum(x) for x in zip(mid, norm)], 180 - rot
    
# Query edge centrality
edgeResp = client.request(url = f'/v2/centrality/{profile}/json', get_params = {}, post_json = {'bbox': neuenheim, 'mode': "edges"})

# dict of nodeId -> coordinate for easier access
locDict = {node['nodeId']: node['coord'] for node in edgeResp['locations']}

for node in edgeResp['locations']:
    nodeId = node['nodeId']
    folium.CircleMarker(color = "green", location = list(reversed(node['coord'])),
                       radius = 5,
                      tooltip = f"id: {nodeId}").add_to(edge_map)

# display centrality on map
for edge in edgeResp['edgeScores']:
    fromId = edge['fromId']
    toId = edge['toId']
    fromCoord = list(reversed(locDict[fromId]))
    toCoord = list(reversed(locDict[toId]))
    mid, rot = midOffset(fromCoord, toCoord)
    folium.PolyLine(locations=[fromCoord, mid, toCoord],
                   tooltip = f"{fromId} -> {toId}, score: {edge['score']}").add_to(edge_map)
    folium.RegularPolygonMarker(location=mid, color='red', stroke=1, fill_color='red', number_of_sides=3, radius=8, rotation=rot).add_to(edge_map)

edge_map